UTS

In [14]:
import numpy as np
import pandas as pd


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
df = pd.read_csv('./drive/MyDrive/Colab Notebooks/uts/tweet_emotions.csv')

df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [56]:
# Case Folding
import nltk
import re, string, unicodedata

def remove_username(text):
    return re.sub(r'@[^\s]+','', text)

def remove_url(text):
    return re.sub(r'http\S+','', text)

def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
response = vectorizer.fit_transform(df)
print(response)

  (0, 2)	1.0
  (1, 1)	1.0
  (2, 0)	1.0


In [64]:
def filtering_lower(text):
    filter= []
    for i in text.split():
        if i.strip().lower() not in sw and i.strip().lower().isalpha():
            filter.append(i.strip().lower())
    return " ".join(filter)

In [65]:
# STEMMER
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text

In [68]:
# EXECUTION

def preprocessing(text):
    # case folding
    text = remove_username(text)
    text = remove_url(text)
    text = remove_special_characters(text, remove_digits=True)
    

In [69]:
def preprocessing(text):
  text = filtering_lower(text)

In [70]:
def preprocessing(text):
  text = stemmer(text)
  return text

In [71]:
df['final'] = df['content'].apply(preprocessing)

df.head()

,tweet_id,sentiment,content,final
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,@tiffanylu i know i wa listenin to bad habit e...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed with a headach ughhhh...waitin on ...
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funer ceremony...gloomi friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!,want to hang out with friend soon!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,@dannycastillo we want to trade with someon wh...


In [72]:
# check data each emotions 
df["sentiment"].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [73]:
# mengubah data kategori menjadi numerik
new_labels = {
    'anger': 0,
    'boredom': 1,
    'empty': 2,
    'enthusiasm': 3,
    'fun': 4,
    'happiness': 5,
    'hate': 6,
    'love': 7,
    'neutral': 8,
    'relief': 9,
    'sadness': 10,
    'surprise': 11,
    'worry': 12
}

#Encode labels
df['sentiment'] = df['sentiment'].map(new_labels)
df.head()

,tweet_id,sentiment,content,final
0,1956967341,2,@tiffanylue i know i was listenin to bad habi...,@tiffanylu i know i wa listenin to bad habit e...
1,1956967666,10,Layin n bed with a headache ughhhh...waitin o...,layin n bed with a headach ughhhh...waitin on ...
2,1956967696,10,Funeral ceremony...gloomy friday...,funer ceremony...gloomi friday...
3,1956967789,3,wants to hang out with friends SOON!,want to hang out with friend soon!
4,1956968416,8,@dannycastillo We want to trade with someone w...,@dannycastillo we want to trade with someon wh...


In [74]:
# Split Data
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()

X = tfidf_vect.fit_transform(df["final"])
y = df["sentiment"]

In [75]:
#Split Data Training dan Data Testing
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [76]:
#Training And Evaluating Model
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score,classification_report

# C-Support Vector Classification
model = SVC()

# Trainingg
model.fit(X,y)

#prediksi data testing
y_pred_test = model.predict(X_test)

#prediksi data training
y_pred_train = model.predict(X_train)  

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Menghitung Precission
precission = precision_score(y_test, y_pred_test, average='weighted',zero_division=0)

print(classification_report(y_test, y_pred_test, target_names=new_labels.keys(),zero_division=0))

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')
print(f'Hasil presisi data test: {precission}')

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        36
     boredom       1.00      0.11      0.20        46
       empty       1.00      0.04      0.07       254
  enthusiasm       1.00      0.01      0.02       219
         fun       1.00      0.47      0.64       533
   happiness       0.80      0.87      0.83      1589
        hate       0.90      0.46      0.61       369
        love       0.83      0.78      0.81      1163
     neutral       0.67      0.94      0.78      2621
      relief       0.96      0.32      0.48       465
     sadness       0.92      0.81      0.86      1538
    surprise       0.99      0.41      0.58       655
       worry       0.73      0.94      0.82      2512

    accuracy                           0.77     12000
   macro avg       0.83      0.47      0.51     12000
weighted avg       0.81      0.77      0.74     12000

Hasil akurasi data train: 0.7670714285714286
Hasil akurasi data test: 0.7673333